In [ ]:
import pandas as pd
import numpy as np
import time
import datetime
import pickle
import random
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2, 3, 4"
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
# logging.basicConfig(level=logging.INFO, filename='log_Model', format=LOG_FORMAT)
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

from IPython.display import display, HTML
def show_df(df):
    display(HTML(df.to_html()))
    
VAL_POINT = 65623


from tools.pos_encoder import *
from tools.time_encoder import *

pickle_path = 'var/build_dataset/df_train_info.pickle'
with open(pickle_path, 'rb') as f:
    df_train_info = pickle.load(f)

In [ ]:
logging.info('start read data')
pickle_path = '../user_data/data_sort_out/train_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_mp = pickle.load(f)
key_list_train, mp_action_train, mp_distance_train, mp_order_train, mp_distance_train_detail = \
train_dataset_mp['key_list'], train_dataset_mp['mp_action'], train_dataset_mp['mp_distance'], train_dataset_mp['mp_order'], train_dataset_mp['mp_distance_detail']

pickle_path = '../user_data/data_sort_out/testA_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_mp = pickle.load(f)
key_list_testA, mp_action_testA, mp_distance_testA, mp_order_testA, mp_distance_testA_detail = \
testA_dataset_mp['key_list'], testA_dataset_mp['mp_action'], testA_dataset_mp['mp_distance'], testA_dataset_mp['mp_order'], testA_dataset_mp['mp_distance_detail']

pickle_path = '../user_data/data_sort_out/train_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_df = pickle.load(f)
df_actions_train, df_couriers_train, df_distances_train, df_orders_train = \
train_dataset_df['df_actions'], train_dataset_df['df_couriers'], train_dataset_df['df_distances'], train_dataset_df['df_orders']
    
pickle_path = '../user_data/data_sort_out/testA_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_df = pickle.load(f)
df_actions_testA, df_couriers_testA, df_distances_testA, df_orders_testA = \
testA_dataset_df['df_actions'], testA_dataset_df['df_couriers'], testA_dataset_df['df_distances'], testA_dataset_df['df_orders']

logging.info('finish read data')

In [ ]:

def build_dict(mp, date, courier, wave_idx):
    if date not in mp:
        mp[date] = {}
    if courier not in mp[date]:
        mp[date][courier] = {}
    if wave_idx not in mp[date][courier]:
         mp[date][courier][wave_idx] = {}
    return mp

def pick_part_features(s, e):
    mp_action_part, mp_distance_part, mp_order_part, mp_couriers_part = {}, {}, {}, {}
    for i in range(s, e):
        date, courier, wave_idx = key_list_train[i]
        mp_action_part = build_dict(mp_action_part, date, courier, wave_idx)
        mp_distance_part = build_dict(mp_distance_part, date, courier, wave_idx)
        mp_order_part = build_dict(mp_order_part, date, courier, wave_idx)        
        mp_action_part[date][courier][wave_idx] = mp_action_train[date][courier][wave_idx]
        mp_distance_part[date][courier][wave_idx] = mp_distance_train_detail[date][courier][wave_idx]
        mp_order_part[date][courier][wave_idx] = mp_order_train[date][courier][wave_idx]
        if courier not in mp_couriers_part:
            mp_couriers_part[courier] = {}
        mp_couriers_part[courier][date] = mp_couriers_features_train[courier][date]
    
    key_list = key_list_train[s : e]
    
    mp_info_dict = {
        'action' : mp_action_part,
        'distance' : mp_distance_part,
        'order' : mp_order_part,
        'couriers' : mp_couriers_part,
    }
    data_info_dict = {
        'know_lens_list' : df_train_info['know_lens_list'].values[s:e],
        'full_lens' : df_train_info['lens'].values[s:e],
        'impossible_idxs_list' : df_train_info['impossible_idxs_list'].values[s:e]
    }    
    
    return key_list, mp_info_dict, data_info_dict

part = 32
len_key_list = len(key_list_train)
s_list, e_list = [], []

for i in range(0, len_key_list, int(len_key_list / part)):
    start, end = i, i + int(len_key_list / part)
    end = min(end, len_key_list)
    logging.info(('%d, %d') %(start,end))
    key_list, mp_info_dict, data_info_dict = pick_part_features(start, end)
    pickle_path = '../user_data/part_train_mp/keylist_%d_%d.pickle' % (start, end)
    with open(pickle_path, 'wb') as f:
        pickle.dump(key_list, f)
        
    pickle_path = '../user_data/part_train_mp/mp_info_dict_%d_%d.pickle' % (start, end)
    with open(pickle_path, 'wb') as f:
        pickle.dump(mp_info_dict, f)
            
    pickle_path = '../user_data/part_train_mp/data_info_dict_%d_%d.pickle' % (start, end)
    with open(pickle_path, 'wb') as f:
        pickle.dump(data_info_dict, f)